# 1. Import the necessary libraries for you to be able to query 4square API

In [39]:
import requests
import json
from getpass import getpass
import os
from dotenv import load_dotenv
load_dotenv()
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
import geopandas as gpd


# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [40]:
where = "Ironhack, Barcelona"
url_geocode = f"https://geocode.xyz/{where}?json=1"
res = requests.get(url_geocode)
res.json()

{'standard': {'addresst': {},
  'statename': {},
  'city': 'Barcelona',
  'prov': 'ES',
  'countryname': 'Spain',
  'postal': {},
  'confidence': '0.266666666666667'},
 'longt': '2.15508',
 'alt': {'loc': {'longt': '2.19312',
   'prov': 'ES',
   'city': 'Barcelona',
   'countryname': 'Spain',
   'postal': '08030',
   'region': {},
   'latt': '41.43566'}},
 'elevation': {},
 'latt': '41.40814'}

In [41]:
import time
def gets_coordinates_from_string (where):
    url_geocode = f"https://geocode.xyz/{where}?json=1"
    res = requests.get(url_geocode)
    
    try:
        return res.json()
    
    except:
        time.sleep(2)
        gets_coordinates_from_string (where)

In [42]:
res = gets_coordinates_from_string ("Ironhack, Barcelona")
res

{'standard': {'addresst': {},
  'statename': {},
  'city': 'Barcelona',
  'prov': 'ES',
  'countryname': 'Spain',
  'postal': {},
  'confidence': '0.266666666666667'},
 'longt': '2.15508',
 'alt': {'loc': {'longt': '2.19312',
   'prov': 'ES',
   'city': 'Barcelona',
   'countryname': 'Spain',
   'postal': '08030',
   'region': {},
   'latt': '41.43566'}},
 'elevation': {},
 'latt': '41.40814'}

In [43]:
lat, lon = res["latt"], res["alt"]["loc"]["longt"]
print(lat,lon)

41.40814 2.19312


### Found the lat & Lon but they are a little bit off... 
### So I just went to google Maps and looked up the exact coordinates

![LINK](https://media2.giphy.com/media/3PAGDqG9ENQF8QHZKt/giphy.gif?cid=ecf05e472kv1z6yjqw7xi4fy8iucik0zfexw0oauyqspu0ta&ep=v1_gifs_search&rid=giphy.gif&ct=g)

In [44]:

iron_lat = 41.39774951659203
iron_lon = 2.1902476452286734
map_1 = Map(location = [iron_lat, iron_lon], zoom_start=15)
ironhack = Marker(location = [iron_lat, iron_lon], tooltip="Ironhack")
ironhack.add_to(map_1) 
map_1


In [45]:
import os
from dotenv import load_dotenv
load_dotenv() # see if I have the correct environment

True

In [46]:
cc = os.getenv("credit_card")
token = os.getenv("token")

In [47]:
def requests_for_foursquare (query, lat, lon, radius=500, limit=1):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [74]:
res = requests_for_foursquare ("coffee", iron_lat, iron_lon, radius=50000, limit=10)
res

{'results': [{'fsq_id': '5b8521d489e490002cefef18',
   'categories': [{'id': 13035,
     'name': 'Coffee Shop',
     'short_name': 'Coffee Shop',
     'plural_name': 'Coffee Shops',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'}},
    {'id': 13065,
     'name': 'Restaurant',
     'short_name': 'Restaurant',
     'plural_name': 'Restaurants',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 567,
   'geocodes': {'main': {'latitude': 41.397168, 'longitude': 2.183123},
    'roof': {'latitude': 41.397168, 'longitude': 2.183123}},
   'link': '/v3/places/5b8521d489e490002cefef18',
   'location': {'address': 'Ausiàs Marc, 151',
    'admin_region': 'Cataluña',
    'country': 'ES',
    'formatted_address': 'Ausiàs Marc, 151, 08013 Barcelona Catalunya',
    'locality': 'Barcelona',
    'postcode': '08013',
    '

In [75]:
names = []
addresses = []
distances = []
postcodes = []

# Iterate through the 'results' in your data
for result in res['results']:
    name = result.get('name', '')
    address = result['location'].get('formatted_address', '')
    distance = result.get('distance', '')
    postcode = result['location'].get('postcode', '')
    names.append(name)
    addresses.append(address)
    distances.append(distance)
    postcodes.append(postcode)

dictionary_places = {"Coffee Shop": names,"Address": addresses,"Distance (in m)": distances,"Postcode": postcodes}

print(dictionary_places)

{'Coffee Shop': ['Three Marks Coffee', "Nømad Roaster's Home", 'Funky Bakers', 'Nømad Coffee Lab & Shop', 'Brew Coffee', 'El Petit Príncep', 'Cafés el Magnífico', 'BRKLYN Torrefaction', 'Federal', 'Dalston Coffee'], 'Address': ['Ausiàs Marc, 151, 08013 Barcelona Catalunya', 'Calle de Pujades, 95 (C. de Badajoz), 08005 Barcelona', 'Carrer de la Diputació, 347, 08009 Barcelona Catalunya', 'Pasatge Sert, 12 (C. de Trafalgar), 08003 Barcelona Catalunya', 'Calle de Roger de Flor, 102, 08013 Barcelona Catalunya', 'Gran Vía de Les Corts Catalanes, 677 (btwn c/ Nàpols & c/ Roger de Flor), 08013 Barcelona Catalunya', 'Argenteria, 64, 08003 Barcelona Catalunya', 'Calle de Bac de Roda, 79 (C. de Pujades), 08005 Barcelona Catalunya', 'Carrer del Taulat, 109, 08005 Barcelona Catalunya', 'Calle de Les Ramelleres, 16, 08001 Barcelona Catalunya'], 'Distance (in m)': [567, 458, 1485, 1466, 1047, 1101, 1754, 1582, 1318, 2344], 'Postcode': ['08013', '08005', '08009', '08003', '08013', '08013', '08003', '

In [76]:
dataframe_coffeeshops = pd.DataFrame(dictionary_places)
dataframe_coffeeshops

,Coffee Shop,Address,Distance (in m),Postcode
0,Three Marks Coffee,"Ausiàs Marc, 151, 08013 Barcelona Catalunya",567,08013
1,Nømad Roaster's Home,"Calle de Pujades, 95 (C. de Badajoz), 08005 Ba...",458,08005
2,Funky Bakers,"Carrer de la Diputació, 347, 08009 Barcelona C...",1485,08009
3,Nømad Coffee Lab & Shop,"Pasatge Sert, 12 (C. de Trafalgar), 08003 Barc...",1466,08003
4,Brew Coffee,"Calle de Roger de Flor, 102, 08013 Barcelona C...",1047,08013
5,El Petit Príncep,"Gran Vía de Les Corts Catalanes, 677 (btwn c/ ...",1101,08013
6,Cafés el Magnífico,"Argenteria, 64, 08003 Barcelona Catalunya",1754,08003
7,BRKLYN Torrefaction,"Calle de Bac de Roda, 79 (C. de Pujades), 0800...",1582,08005
8,Federal,"Carrer del Taulat, 109, 08005 Barcelona Catalunya",1318,08005
9,Dalston Coffee,"Calle de Les Ramelleres, 16, 08001 Barcelona C...",2344,08001


# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [77]:
res = requests_for_foursquare ("Coffee", lat, lon, radius=500, limit=5)


In [78]:
names = []
addresses = []
distances = []
postcodes = []

# Iterate through the 'results' in your data
for result in res['results']:
    name = result.get('name', '')
    address = result['location'].get('formatted_address', '')
    distance = result.get('distance', '')
    postcode = result['location'].get('postcode', '')
    names.append(name)
    addresses.append(address)
    distances.append(distance)
    postcodes.append(postcode)

dictionary_5_closest_places = {"Coffee Shop": names,"Address": addresses,"Distance (in m)": distances,"Postcode": postcodes}

print(dictionary_5_closest_places)

{'Coffee Shop': ['Chök', 'Buenas Migas', 'Buenas Migas', 'Manolo Bakes', 'Clot'], 'Address': ['Avinguda Diagonal, 208 (Westfield Glòries), 08018 Barcelona Catalunya', 'Av. Diagonal, 208, 08018 Barcelona Catalunya', 'Avenida Diagonal, 197, 08018 Barcelona Catalunya', 'Avinguda Diagonal, 208 (Westfield Glòries), 08018 Barcelona Catalunya', 'Plaza Font I Sagué, 3 (Carrer del Clot), 08018 Barcelona Catalunya'], 'Distance (in m)': [316, 333, 462, 325, 380], 'Postcode': ['08018', '08018', '08018', '08018', '08018']}


In [79]:
dataframe_coffeeshops = pd.DataFrame(dictionary_5_closest_places)
dataframe_coffeeshops

,Coffee Shop,Address,Distance (in m),Postcode
0,Chök,"Avinguda Diagonal, 208 (Westfield Glòries), 08...",316,08018
1,Buenas Migas,"Av. Diagonal, 208, 08018 Barcelona Catalunya",333,08018
2,Buenas Migas,"Avenida Diagonal, 197, 08018 Barcelona Catalunya",462,08018
3,Manolo Bakes,"Avinguda Diagonal, 208 (Westfield Glòries), 08...",325,08018
4,Clot,"Plaza Font I Sagué, 3 (Carrer del Clot), 08018...",380,08018


# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [80]:
def foursquare_places (venue, coordinates):
    pass

# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [81]:
# your code here

# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [82]:
# your code here